<h1 align="center"><b>Segmenting and Clustering Neighborhoods in Toronto</b></h1>
<h1 align="center"><b>PART 3</b></h1>

<h5 align="center"><b>The processes of PART 1 and PART 2 will be repeated in this Notebook. PART 3 follows after PART 1 and PART 2</b></h5>

 #### 1. Install and import the necessary libraries

Install the necessary libraries [Beautiful Soup, lxml]

In [1]:
# Install Beautiful Soup 4
!conda install -c conda-forge beautifulsoup4 --yes

# Install lxml parser
!conda install -c conda-forge lxml --yes

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



Import the necessary libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

 #### 2. Scrape the Toronto post codes Wikipedia page with Beautiful Soup

Read the Toronto post codes wikipedia page with Beautiful Soup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

# Get the table with the postal codes [class = 'wikitable sortable']
table = soup.find('table', class_='wikitable sortable')

Convert the postal codes table to a pandas Data Frame

In [4]:
postal_codes_initial_df = pd.read_html(str(table))[0]

Give the columns the required names

In [5]:
postal_codes_initial_df.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
print(postal_codes_initial_df.head(10), '\n')
print('Initial Toronto postal codes dataframe shape: ', postal_codes_initial_df.shape, '\n')

  PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront
5        M5A  Downtown Toronto       Regent Park
6        M6A        North York  Lawrence Heights
7        M6A        North York    Lawrence Manor
8        M7A      Queen's Park      Not assigned
9        M8A      Not assigned      Not assigned 

Initial Toronto postal codes dataframe shape:  (288, 3) 



 #### 3. Process the initial Toronto post codes Data Frame

Drop the rows with the 'Borough' value 'Not Assigned'

In [6]:
postal_codes_initial_df.drop(postal_codes_initial_df[postal_codes_initial_df['Borough'] == 'Not assigned'].index, axis=0, inplace=True)

Replace the 'Not assigned' value in 'Neighborhood' with the corresponding value of 'Borough'

In [7]:
postal_codes_initial_df.loc[postal_codes_initial_df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = postal_codes_initial_df.loc[postal_codes_initial_df['Neighborhood'] == 'Not assigned']['Borough']
postal_codes_initial_df.head(10) # Notice the Queen's Park Neighborhood

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Create a new Data Frame to combine the neighborhoods of the same postal code

In [8]:
postal_codes_grouped_df = pd.DataFrame(postal_codes_initial_df.groupby('PostalCode')['Neighborhood'].apply(lambda tags: ', '.join(tags))).reset_index()

Check that the dataframe was created successfully

In [9]:
postal_codes_grouped_df.head(10)

,PostalCode,Neighborhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
5,M1J,Scarborough Village
6,M1K,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,"Clairlea, Golden Mile, Oakridge"
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,"Birch Cliff, Cliffside West"


Now merge the initial and combined neighborhoods data frames to the final data frame

In [10]:
toronto_postal_codes_df = pd.merge(postal_codes_initial_df, postal_codes_grouped_df, how='inner', on=['PostalCode', 'PostalCode'])

# Drop the single neighborhood column
toronto_postal_codes_df.drop('Neighborhood_x', axis=1, inplace=True)

# Now drop the created duplicate rows
toronto_postal_codes_df.drop_duplicates(inplace=True)

# Rename the created Neighborhood_y to Neighborhood
toronto_postal_codes_df.rename(columns={'Neighborhood_y': 'Neighborhood'}, inplace=True)

# Reset the data frame index
toronto_postal_codes_df.reset_index(drop=True, inplace=True)

Check an example from the instruction where one postal code has many neighborhoods

In [11]:
# Set the pandas dataframe display.max_colwidth to -1 so that the full data frame columns are shown
pd.set_option('display.max_colwidth', -1)
toronto_postal_codes_df[toronto_postal_codes_df['PostalCode'] == 'M5V']

,PostalCode,Borough,Neighborhood
87,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara"


Examine the final dataframe

In [12]:
toronto_postal_codes_df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Finally print the shape of the final data frame

In [13]:
print('The final dataframe has {} rows.'.format(toronto_postal_codes_df.shape[0]))

The final dataframe has 103 rows.


<h1 align="center"><b>PART 2</b></h1>

#### 1. Read the geospatial coordinates file

I tried to use the geocoder package but it was not working properly. So I downloaded 'Geospatial_Coordinates.csv' (manually on my PC - I don't have wget installed)

Read the geospatial coordinates file into a data frame

In [14]:
toronto_latlng_df = pd.read_csv('Geospatial_Coordinates.csv')

# Check the created dataframe
toronto_latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two data neighborhood and geospatial coordinates data frame

In [15]:
# Merge data frames
toronto_postal_latlng_df = pd.merge(toronto_postal_codes_df, toronto_latlng_df, how='inner', left_on='PostalCode', right_on='Postal Code')
toronto_postal_latlng_df.drop('Postal Code', axis=1, inplace=True)
toronto_postal_latlng_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


<h1 align="center"><b>PART 3</b></h1>

In this part of the assignment we will segment the boroughs of Toronto and explore them based ON POSTAL CODES INSTEAD OF NEIGHBORHOODS, since we only have geospatial data corresponding to postal codes. As we saw in the previous parts of the assignment, one postal code may correspond to many neighborhoods. The data is contained in the data frame that was previously created: <b>toronto_postal_latlng_df<b>

 #### 1. Install and import the necessary libraries

Install necessary libraries [geopy, folium]

In [16]:
!conda install -c conda-forge geopy --yes

#!conda install -c conda-forge folium=0.5.0
!conda install -c conda-forge folium --yes

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



Import libraries

In [17]:
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


#### 2. Create a map of Toronto with boroughs superimposed on top.

Use geopy library to get the latitude and longitude values of Toronto.<br>
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer

In [18]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Now, let's create the map of Toronto and its boroughs

In [19]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postal_code, neighborhoods in zip(toronto_postal_latlng_df['Latitude'],
                                                         toronto_postal_latlng_df['Longitude'],
                                                         toronto_postal_latlng_df['Borough'],
                                                         toronto_postal_latlng_df['PostalCode'],
                                                         toronto_postal_latlng_df['Neighborhood']):
    label = '{}, {}, {}'.format(borough, postal_code, neighborhoods)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)
    
map_toronto

The map will not display on github, so I added it as an html image in a markdown cell.

<img src="map_toronto.png">

#### 3. Explore Neighborhoods (Postal Codes) in Toronto

In [20]:
# Foursquare credentials and version
CLIENT_ID = 'SJU0OHQPT1C04JSLXW30XLXTVV4M014ULW12OJ0U3ZDDWQCH' # your Foursquare ID
CLIENT_SECRET = 'KQOIWUEOFVWJDRKRI5CX5G5XX4T0XVTRKJSFDYVOBKSCYSYD' # your Foursquare Secret
VERSION = '20190620' # Foursquare API version

radius = 500
LIMIT = 100

 Create a function to get the venues of all the neighborhoods (postal codes) in Toronto

In [21]:
def getNearbyVenues(postal_codes, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postal_code, neighborhood, lat, lng in zip(postal_codes, neighborhoods, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code,
            neighborhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                  'Neighborhood',
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each postal code and create a new dataframe called __*toronto_venues*__

In [22]:
toronto_venues = getNearbyVenues(postal_codes=toronto_postal_latlng_df['PostalCode'],
                                 neighborhoods=toronto_postal_latlng_df['Neighborhood'],
                                 latitudes=toronto_postal_latlng_df['Latitude'],
                                 longitudes=toronto_postal_latlng_df['Longitude'])

In [23]:
print(toronto_venues.shape)
toronto_venues.head(10)

(2259, 8)


,PostalCode,Neighborhood,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,M4A,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,M4A,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,M5A,"Harbourfront, Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,M5A,"Harbourfront, Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,M5A,"Harbourfront, Regent Park",43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center


In [24]:
toronto_venues.groupby(['PostalCode', 'Neighborhood']).count().head(20)

,,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,Neighborhood,,,,,,
M1B,"Rouge, Malvern",2,2,2,2,2,2
M1C,"Highland Creek, Rouge Hill, Port Union",2,2,2,2,2,2
M1E,"Guildwood, Morningside, West Hill",9,9,9,9,9,9
M1G,Woburn,4,4,4,4,4,4
M1H,Cedarbrae,8,8,8,8,8,8
M1J,Scarborough Village,1,1,1,1,1,1
M1K,"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4
M1L,"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
M1M,"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3


Check how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} unique venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 280 unique venue categories.


#### 4. Analyze Each Neighborhood (Postal Code) in Toronto

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add postal code, neighborhood column back to dataframe
# toronto_onehot[['PostalCode', 'Neighborhood']] = toronto_venues[['PostalCode', 'Neighborhood']]
toronto_onehot[['PostalCode', 'PostNeighborhood']] = toronto_venues[['PostalCode', 'Neighborhood']]
# toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# Move postNeighborhood column to the first column

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# Move PostalCode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# toronto_onehot.head()
# toronto_onehot.columns
# # toronto_onehot[['PostalCode', 'Postal Code Latitude']].head(10)
# fixed_columns
toronto_onehot.head(10)

,PostalCode,PostNeighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse St

In [27]:
toronto_onehot.shape

(2259, 282)

Ggroup rows by PostalCode and PostNeighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby(['PostalCode', 'PostNeighborhood']).mean().reset_index()
toronto_grouped.head(10)

,PostalCode,PostNeighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse St

In [29]:
toronto_grouped.shape

(100, 282)

Print each neighborhood along with the top 5 most common venues

__Create a data frame that contains PostalCode, PostNeighborhood and the corresponding 10 most common venues__

Function to sort the venues in descending frequency order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    # The first two columns will be: PostalCode and PostNeighborhood
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode', 'PostNeighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted[['PostalCode', 'PostNeighborhood']] = toronto_grouped[['PostalCode', 'PostNeighborhood']]
# neighborhoods_venues_sorted['PostNeighborhood'] = toronto_grouped['PostNeighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,PostalCode,PostNeighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M1C,"Highland Creek, Rouge Hill, Port Union",Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,M1E,"Guildwood, Morningside, West Hill",Intersection,Spa,Electronics Store,Pizza Place,Breakfast Spot,Tech Startup,Medical Center,Rental Car Location,Mexican Restaurant,Dog Run
3,M1G,Woburn,Coffee Shop,Convenience Store,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Cedarbrae,Hakka Restaurant,Bakery,Lounge,Caribbean Restaurant,Athletics & Sports,Bank,Fried Chicken Joint,Thai Restaurant,Yoga Studio,Drugstore


### 5. Cluster Neighborhoods (Postal Codes) in Toronto (K-Means Clustering)

Run *k*-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('PostNeighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 2,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 4, 0, 1, 0, 0, 4, 2, 0, 2, 0, 0])

Create a new dataframe that includes the cluster as well as the top 10 venues for each postal code (neighborhood).

In [33]:
# add clustering labels
# toronto_postal_codes_df
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged_df = toronto_postal_latlng_df.copy()
neighborhoods_venues_sorted.rename(columns={'PostNeighborhood': 'Neighborhood'}, inplace=True)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged_df = toronto_merged_df.join(neighborhoods_venues_sorted.set_index(['PostalCode', 'Neighborhood']), on=['PostalCode', 'Neighborhood'])

toronto_merged_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,Hockey Arena,Coffee Shop,Portuguese Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Empanada Restaurant,Electronics Store,Dessert Shop
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Café,Farmers Market
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Café,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,0.0,Fast Food Restaurant,Pizza Place,Pet Store,Gastropub,Café,Bank,Athletics & Sports,Intersection,Breakfast Spot,Gym / Fitness Center
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Tea Room,Theater,Ramen Restaurant,Pizza Place


Some of the values in 'Cluster Labels' are __NaN__

These rows will be dropped

In [34]:
toronto_merged_df.shape

(103, 16)

Check how many 'Cluster Labels' are __NaN__

In [35]:
toronto_merged_df[toronto_merged_df['Cluster Labels'].isnull()]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Three 'Cluster Labels' are __NaN__

Drop the three rows with __NaN__ in 'Cluster Labels'

In [36]:
toronto_merged_df.dropna(inplace=True)
toronto_merged_df.shape

(100, 16)

Convert the values of 'Cluster Labels' into integers

In [37]:
toronto_merged_df['Cluster Labels']= toronto_merged_df['Cluster Labels'].astype(int)
toronto_merged_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Intersection,Hockey Arena,Coffee Shop,Portuguese Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Empanada Restaurant,Electronics Store,Dessert Shop
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Café,Farmers Market
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
7,M3B,North York,Don Mills North,43.745906,-79.352188,0,Café,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,0,Fast Food Restaurant,Pizza Place,Pet Store,Gastropub,Café,Bank,Athletics & Sports,Intersection,Breakfast Spot,Gym / Fitness Center
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Tea Room,Theater,Ramen Restaurant,Pizza Place
10,M6B,North York,Glencairn,43.709577,-79.445073,2,Park,Pizza Place,Japanese Restaurant,Pub,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


Visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, code, poi, cluster in zip(toronto_merged_df['Latitude'], toronto_merged_df['Longitude'], toronto_merged_df['PostalCode'], toronto_merged_df['Neighborhood'], toronto_merged_df['Cluster Labels']):
    label = folium.Popup(str(code) + ' - ' +str(poi) + ' >> Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The map will not display on github, so I added it as an html image in a markdown cell.

<img src="map_clusters.png">

## 6. Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, a name will be assigned to each cluster.

#### Cluster 1 [Cluster Labels = 0]

In [39]:
toronto_merged_df.loc[toronto_merged_df['Cluster Labels'] == 0, toronto_merged_df.columns[[1] + list(range(5, toronto_merged_df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Intersection,Hockey Arena,Coffee Shop,Portuguese Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Empanada Restaurant,Electronics Store,Dessert Shop
2,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Café,Farmers Market
3,North York,0,Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,Queen's Park,0,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
6,Scarborough,0,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
7,North York,0,Café,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,East York,0,Fast Food Restaurant,Pizza Place,Pet Store,Gastropub,Café,Bank,Athletics & Sports,Intersection,Breakfast Spot,Gym / Fitness Center
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Tea Room,Theater,Ramen Restaurant,Pizza Place
12,Scarborough,0,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
13,North York,0,Gym,Asian Restaurant,Beer Store,Coffee Shop,Dim Sum Restaurant,Bike Shop,Japanese Restaurant,Italian Restaurant,Restaurant,Chinese Restaurant


#### Cluster 2 [Cluster Labels = 1]

In [40]:
toronto_merged_df.loc[toronto_merged_df['Cluster Labels'] == 1, toronto_merged_df.columns[[1] + list(range(5, toronto_merged_df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,1,Bank,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Diner


#### Cluster 3 [Cluster Labels = 2]

In [42]:
toronto_merged_df.loc[toronto_merged_df['Cluster Labels'] == 2, toronto_merged_df.columns[[1] + list(range(5, toronto_merged_df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
10,North York,2,Park,Pizza Place,Japanese Restaurant,Pub,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
21,York,2,Park,Pharmacy,Market,Fast Food Restaurant,Women's Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Dim Sum Restaurant
35,East York,2,Park,Coffee Shop,Convenience Store,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,North York,2,Park,Airport,Other Repair Shop,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
61,Central Toronto,2,Park,Bus Line,Swim School,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
64,York,2,Park,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
66,North York,2,Park,Convenience Store,Bank,Bar,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
68,Central Toronto,2,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
77,Etobicoke,2,Pizza Place,Park,Mobile Phone Shop,Bus Line,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop


#### Cluster 4 [Cluster Labels = 3]

In [43]:
toronto_merged_df.loc[toronto_merged_df['Cluster Labels'] == 3, toronto_merged_df.columns[[1] + list(range(5, toronto_merged_df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,3,Cafeteria,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,Diner


#### Cluster 5 [Cluster Labels = 4]

In [44]:
toronto_merged_df.loc[toronto_merged_df['Cluster Labels'] == 4, toronto_merged_df.columns[[1] + list(range(5, toronto_merged_df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4,Baseball Field,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Diner
101,Etobicoke,4,Baseball Field,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Diner
